# 뉴스 카테고리 다중분류 - 번외

## 프로젝트 개요
텐서플로 데이터셋 중 하나인 '로이터 뉴스 주제 분류'에 대한 SVM 모델을 구축하고 테스트합니다.
GLoVe 모델을 이용하여 토큰 임베딩을 구축하고, 이것을 이용해 유의어를 교체하는 방식의 텍스트 증강을 시행한 데이터로 2차 테스트하고, 텍스트 증강전후의 성능을 비교합니다.

## 코드 구성
코드는 크게 세 부분으로 나뉩니다.

- 데이터 전처리 (함수)

로이터 뉴스 데이터에 대해서, 텍스트를 딕셔너리로 바꾸고, 딕셔너리를 다시 디코딩하는 기능을 갖춘 함수입니다.

- 데이터 증강을 위한 GloVe 모델 구축

GloVe 모델로 단어 임베딩을 만들고, 이를 활용한 유의어 사전으로 단어를 교체해서 텍스트를 증강하고 이를 전처리하는 부분입니다.

- 모델 구축 (함수)

SVM 모델과 하이퍼 파라미터 설정을 하는 부분입니다.

- 모델 실행결과 저장 및 시각화

In [7]:
# 셸 환경에서 명령어 실행
!pip install gensim

In [10]:
import numpy as np
import time
from tensorflow.keras.datasets import reuters
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors

# 로이터 데이터셋을 로드하고 전처리하는 함수
def load_and_preprocess_data(num_words=None):
    (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=num_words, test_split=0.2)
    
    word_index = reuters.get_word_index(path="reuters_word_index.json")
    # 인덱스에서 단어로의 매핑을 생성합니다.
    index_to_word = {index + 3: word for word, index in word_index.items()}
    # 특수 토큰을 추가합니다.
    for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
        index_to_word[index] = token
    
    # 훈련 데이터와 테스트 데이터를 텍스트로 디코딩합니다.
    decoded_x_train = [' '.join([index_to_word.get(i, '?') for i in sequence]) for sequence in x_train]
    decoded_x_test = [' '.join([index_to_word.get(i, '?') for i in sequence]) for sequence in x_test]
    
    return decoded_x_train, y_train, decoded_x_test, y_test


In [11]:
# GloVe 모델을 Word2Vec 포맷으로 변환한 후 로드하는 함수
def load_glove_model(glove_input_file, word2vec_output_file):
    glove2word2vec(glove_input_file, word2vec_output_file)
    model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
    return model

# TF-IDF와 GloVe 모델을 사용하여 단어를 교체함으로써 데이터를 증강하는 함수
def augment_data_by_replacing_words(texts, glove_model, tfidf_vectorizer):
    augmented_texts = []
    # TF-IDF 벡터라이저를 사용하여 텍스트를 변환합니다.
    tfidf_matrix = tfidf_vectorizer.transform(texts)
    
    for text, row in zip(texts, tfidf_matrix):
        words = text.split()
        # TF-IDF 점수가 가장 낮은 단어의 인덱스를 찾습니다.
        min_tfidf_idx = row.argmin()
        # 교체할 단어를 선택합니다.
        replace_word = tfidf_vectorizer.get_feature_names_out()[min_tfidf_idx]

        # GloVe 모델을 사용하여 유사한 단어를 찾습니다.
        synonym = glove_model.most_similar(replace_word)
        if synonym:
            synonym = synonym[0][0]  # 가장 유사한 단어를 선택합니다.
            # 선택된 단어로 교체합니다.
            words = [synonym if word == replace_word else word for word in words]
        augmented_texts.append(' '.join(words))
    
    return augmented_texts

# 데이터 전처리 및 증강을 수행하고, 모델 실행 결과를 반환하는 함수
def preprocess_augment_and_run_model(num_words, augment, glove_path, train_glove, model_func):
    # 데이터 로드 및 전처리
    x_train, y_train, x_test, y_test = load_and_preprocess_data(num_words)
    
    # 증강 옵션에 따라 GloVe 모델을 로드하고 데이터 증강을 수행
    if augment:
        tokenized_sentences = [text.lower().split() for text in x_train]
        glove_model = initialize_or_load_glove_model(tokenized_sentences, glove_path, train=train_glove)
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_vectorizer.fit(x_train)
        x_train_augmented = augment_data_by_replacing_words(x_train, glove_model, tfidf_vectorizer)
        x_test_augmented = augment_data_by_replacing_words(x_test, glove_model, tfidf_vectorizer)
    else:
        x_train_augmented = x_train
        x_test_augmented = x_test

    # 모델 설정 및 그리드 서치 실행
    model, params = model_func()
    results = run_grid_search(model, params, x_train_augmented, y_train, x_test_augmented, y_test)
    
    return results

In [16]:
from sklearn.svm import SVC

# SVM 설정 함수
def set_model_and_params_svm():
    model = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', SVC()),
    ])
    params = {
        'clf__C': [0.1, 1.0, 10.0],
        'clf__kernel': ['linear', 'rbf'],
    }
    return model, params

# 그리드 서치 실행 및 결과 저장 함수
def run_grid_search(model, params, x_train, y_train, x_test, y_test):
    start_time = time.time()  # 시작 시간 기록
    grid_search = GridSearchCV(model, params, n_jobs=-1, cv=5)
    grid_search.fit(x_train, y_train)
    best_model = grid_search.best_estimator_
    test_score = best_model.score(x_test, y_test)
    end_time = time.time()  # 종료 시간 기록
    elapsed_time = end_time - start_time  # 경과 시간 계산
    return {
        'best_params': grid_search.best_params_,
        'test_score': test_score,
        'elapsed_time': elapsed_time  # 경과 시간 추가
    }

In [13]:
# 결과 시각화 함수 (수정)
def plot_results(results):
    # 결과를 정렬하여 표시합니다.
    sorted_labels = sorted(results.keys(), key=lambda x: (results[x]['test_score'], results[x]['elapsed_time']), reverse=True)
    
    # Test Accuracy와 실행 시간을 분리하여 리스트로 만듭니다.
    labels = []
    test_scores = []
    times = []
    for label in sorted_labels:
        labels.append(label.replace('_', ' ').title())  # 레이블을 보기 좋게 처리합니다.
        test_scores.append(results[label]['test_score'])
        times.append(results[label]['elapsed_time'])

    # 그래프 사이즈를 조정합니다.
    fig, ax1 = plt.subplots(figsize=(10, 8))

    # Test Accuracy 막대 그래프를 그립니다.
    y_positions = np.arange(len(labels))
    bars = ax1.barh(y_positions, test_scores, color='skyblue', label='Test Accuracy')
    ax1.set_yticks(y_positions)
    ax1.set_yticklabels(labels)
    ax1.invert_yaxis()  # 레이블을 높은 정확도가 위로 오도록 뒤집습니다.
    ax1.set_xlabel('Test Accuracy', color='skyblue')
    ax1.tick_params(axis='x', labelcolor='skyblue')

    # 막대 그래프에 정확도 값을 표시합니다.
    for bar, score in zip(bars, test_scores):
        ax1.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, 
                 '{0:.2f}'.format(score), 
                 va='center', ha='right', color='blue', fontsize=8)

    # 실행 시간 라인 그래프를 그립니다.
    ax2 = ax1.twiny()  # x 축을 공유하는 새 축을 생성합니다.
    points = ax2.plot(times, y_positions, 'o', color='salmon', label='Time (s)')
    ax2.set_xlabel('Time (s)', color='salmon')
    ax2.tick_params(axis='x', labelcolor='salmon')

    # 라인 그래프에 실행 시간 값을 표시합니다.
    for time, pos in zip(times, y_positions):
        ax2.text(time, pos, '{0:.2f}s'.format(time), 
                 va='center', ha='left', color='red', fontsize=8)

    plt.title('Model Comparison: Accuracy and Execution Time')
    fig.tight_layout()  # 그래프 레이아웃을 조정합니다.
    plt.show()



In [14]:
# Main execution function
def main():
    num_words = 10000
    glove_input_file = 'glove.6B.100d.txt'  # 실제 경로로 변경해야 합니다.
    word2vec_output_file = 'glove.6B.100d.word2vec.txt'  # 변환된 파일의 경로

    # 데이터 증강 전 모델 실행 결과
    results_before_augmentation = preprocess_augment_and_run_model(
        num_words, augment=False, glove_path=glove_input_file, train_glove=False,
        model_func=set_model_and_params_svm
    )
    
    # 데이터 증강 후 모델 실행 결과
    results_after_augmentation = preprocess_augment_and_run_model(
        num_words, augment=True, glove_path=glove_input_file, train_glove=False,
        model_func=set_model_and_params_svm
    )
    
    # 결과 시각화
    all_results = {
        'before_augmentation': results_before_augmentation,
        'after_augmentation': results_after_augmentation
    }
    plot_results(all_results)

In [ ]:
# Run the main function
if __name__ == '__main__':
    main()